In [63]:
from __future__ import annotations
from rdflib import Graph, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, OWL, XSD, FOAF, DCTERMS
import sbc_tools as sbc
from typing import List, Tuple, Dict, Optional
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import numpy as np
import math
import torch
from pydantic import BaseModel
import copy
import random
import heapq


In [18]:
graph = sbc.load("folktales.ttl")

In [19]:
ONT = Namespace("https://rbc.org/ontology/")
RES = Namespace("https://rbc.org/resources/")

WD = Namespace("http://www.wikidata.org/entity/")		# elementos
WDT = Namespace("http://www.wikidata.org/prop/direct/")	# propiedades
SCHEMA = Namespace("https://schema.org/")

SEM = Namespace("http://semanticweb.cs.vu.nl/2009/11/sem/")
REL = Namespace("http://purl.org/vocab/relationship/")
PEARL = Namespace("https://www.gsi.upm.es/ontologies/pearl/")

In [98]:
class LocalSemanticSimilarityCalculator:
	"""
	Calculadora de similitud semántica que trabaja con ontologías locales
	usando consultas SPARQL sobre el grafo RDF local.

	Similar a SemanticSimilarityCalculator pero ejecuta SPARQL localmente
	en lugar de consultar endpoints remotos.
	"""

	def __init__(self, graph: Graph):
		"""
		Inicializa el calculador con un grafo RDF local

		Args:
			graph: objeto rdflib.Graph con la ontología cargada
		"""
		self.graph = graph
		self.cache = {}

		print(f"Calculador inicializado con {len(self.graph)} triples")
		print(f"Usando consultas SPARQL sobre grafo local\n")

	def execute_query(self, query: str):
		"""Ejecuta consulta SPARQL sobre el grafo local con caché"""
		cache_key = hash(query)
		if cache_key in self.cache:
			return self.cache[cache_key]

		try:
			results = self.graph.query(query)
			result_list = list(results)
			self.cache[cache_key] = result_list
			return result_list
		except Exception as e:
			print(f"Error en consulta SPARQL: {e}")
			return []
		
	def get_class_instances(self, class_id: str):
		query = f"""
		PREFIX rdfs: <{RDFS}>
		PREFIX rdf: <{RDF}>
		PREFIX ont: <{ONT}>

		SELECT DISTINCT ?instance
		WHERE {{
			?subClass rdfs:subClassOf* ont:{class_id} .
  			?instance rdf:type ?subClass .
		}}
		"""

		results = self.execute_query(query)

		if results:
			return [str(result.instance) for result in results]
		return []
		# if results:
		# 	row = results[0]
		# 	lcs_uri = str(row.lcs)
		# 	lcs_qid = lcs_uri.split('/')[-1]
		# 	lcs_label = str(row.lcsLabel) if row.lcsLabel else "Unknown"
		# 	return lcs_qid, lcs_label
		# return None, None

	def get_post_events_number(self, instance_uri: str):
		query = f"""
		PREFIX rdfs: <{RDFS}>
		PREFIX rdf: <{RDF}>
		PREFIX ont: <{ONT}>

		SELECT (COUNT(DISTINCT ?instance) AS ?number)
		WHERE {{
			<{instance_uri}> ont:postEvent ?instance .
		}}
		"""

		results = self.execute_query(query)
		if results:
			return int(results[0].number)
		return 0
	
	def get_post_event_instances(self, instance_uri: str, exclude_list: List[str] = []):
		filter_clause = ""
		if len(exclude_list) > 0:
			exclude_uris = ", ".join(f"<{uri}>" for uri in exclude_list)
			filter_clause = f"FILTER(?instance NOT IN ({exclude_uris}))"

		query = f"""
		PREFIX rdfs: <{RDFS}>
		PREFIX rdf: <{RDF}>
		PREFIX ont: <{ONT}>

		SELECT DISTINCT ?instance
		WHERE {{
			<{instance_uri}> ont:postEvent ?postEventInstance .
			?postEventInstance rdf:type ?class .
			?subClass rdfs:subClassOf* ?class .
			?instance rdf:type ?subClass .

			{filter_clause}
		}}
		"""

		results = self.execute_query(query)

		if results:
			return [str(result.instance) for result in results]
		return []
	
	def get_all_events(self):
		query = f"""
		PREFIX rdfs: <{RDFS}>
		PREFIX rdf: <{RDF}>
		PREFIX ont: <{ONT}>

		SELECT DISTINCT ?instance
		WHERE {{
			?instance rdf:type ?class .
			?class rdfs:subClassOf+ ont:Event .
		}}
		"""

		results = self.execute_query(query)

		if results:
			return [str(result.instance) for result in results]
		return []
		
semantic_sim = LocalSemanticSimilarityCalculator(graph)

Calculador inicializado con 1105 triples
Usando consultas SPARQL sobre grafo local



In [52]:
events = ["event/the_three_little_pigs/villain_fails_sturdy_house", "event/cinderella/wedding_and_throne", "event/the_hare_and_the_tortoise/hero_passing_antagonist"]

# print(events)

event_uris = [URIRef(RES+event) for event in events]

# print(event_uris)

event_names = [event.split("/")[-1] for event in events]

# print(event_names)

In [ ]:
semantic_sim.get_post_events_number(event_uris[2])

semantic_sim.get_post_event_instances(event_uris[2])

['https://rbc.org/resources/event/cinderella/hero_leaves_ball',
 'https://rbc.org/resources/event/cinderella/soulmate_searches_for_hero',
 'https://rbc.org/resources/event/the_hare_and_the_tortoise/antagonist_rushes_to_finish_line',
 'https://rbc.org/resources/event/the_hare_and_the_tortoise/hero_wins_race',
 'https://rbc.org/resources/event/the_three_little_pigs/villain_fails_sturdy_house',
 'https://rbc.org/resources/event/cinderella/wedding_and_throne',
 'https://rbc.org/resources/event/cinderella/soulmate_reveals_hero_true_identity']

In [49]:
semantic_sim.get_class_instances("Conflict")

['https://rbc.org/resources/event/the_hare_and_the_tortoise/antagonist_mocking_hero',
 'https://rbc.org/resources/event/the_hare_and_the_tortoise/antagonist_naps_during_race',
 'https://rbc.org/resources/event/cinderella/magical_helper_grants_wish',
 'https://rbc.org/resources/event/cinderella/hero_meets_his_soulmate',
 'https://rbc.org/resources/event/the_hare_and_the_tortoise/hero_passing_antagonist',
 'https://rbc.org/resources/event/cinderella/villains_insult_hero',
 'https://rbc.org/resources/event/the_three_little_pigs/villain_attacks_flimsy_house',
 'https://rbc.org/resources/event/the_three_little_pigs/villain_attacks_medium_resistant_house']

In [99]:
semantic_sim.get_all_events()

['https://rbc.org/resources/event/cinderella/hero_attends_ball',
 'https://rbc.org/resources/event/the_hare_and_the_tortoise/hero_steady_progress',
 'https://rbc.org/resources/event/cinderella/hero_works_hard',
 'https://rbc.org/resources/event/the_hare_and_the_tortoise/challenge_issued',
 'https://rbc.org/resources/event/the_hare_and_the_tortoise/challenge_accepted',
 'https://rbc.org/resources/event/the_hare_and_the_tortoise/race_begins',
 'https://rbc.org/resources/event/the_three_little_pigs/build_flimsy_house',
 'https://rbc.org/resources/event/the_three_little_pigs/build_medium_resistant_house',
 'https://rbc.org/resources/event/the_three_little_pigs/build_sturdy_house',
 'https://rbc.org/resources/event/the_hare_and_the_tortoise/antagonist_mocking_hero',
 'https://rbc.org/resources/event/the_hare_and_the_tortoise/antagonist_naps_during_race',
 'https://rbc.org/resources/event/cinderella/magical_helper_grants_wish',
 'https://rbc.org/resources/event/cinderella/hero_meets_his_soul

In [114]:
class Query(BaseModel):
	initial_event_type: str
	
class Node(BaseModel):
	parent: Optional[Node] = None
	f: float = 0
	g: float = 0
	h: float = 0
	events: List[str] = []
	
	def is_goal(self, max_events: int):
		event_count = len(self.events)

		if event_count >= max_events:
			return True
		
		if event_count > 0:
			last_event = self.events[-1]
			n_post_events = semantic_sim.get_post_events_number(last_event)
			return n_post_events <= 0
		
		return False
	
	def get_event_names(self):
		return [event.split("/")[-1] for event in self.events]

In [101]:
def state_similarity(node: Node, query: Query):
    return random.uniform(0, 1)

In [119]:
class ConstructiveAdaptation:
	graph: Graph
	semantic_sim: LocalSemanticSimilarityCalculator
	max_events: int
	
	def __init__(self, graph: Graph, semantic_sim: LocalSemanticSimilarityCalculator, max_events):
		self.graph = graph
		self.semantic_sim = semantic_sim
		self.max_events = max_events

	def generate(self, query: Query):
		open_heap: List[Tuple[float, int, Node]] = []
		counter = 0

		initial_candidates = self.semantic_sim.get_class_instances(query.initial_event_type)
		if not initial_candidates:
			# fallback: any event as a starter
			initial_candidates = self.semantic_sim.get_all_events()

		for candidate in initial_candidates:
			node = Node(events=[candidate], parent=None, g=0)
			node.h = 1 - state_similarity(node, query)
			node.f = node.g + node.h
			heapq.heappush(open_heap, (node.f, counter, node))
			counter += 1
			print(f"Initial node added: events={node.get_event_names()}, g={node.g:.2f}, h={node.h:.2f}, f={node.f:.2f}")

		while open_heap:
			_, _, node = heapq.heappop(open_heap)
			print(f"Expanding node: events={node.get_event_names()}, g={node.g:.2f}, h={node.h:.2f}, f={node.f:.2f}")

			if node.is_goal(self.max_events):
				print(f"Goal reached: {node.get_event_names()}")
				return node

			last_event = node.events[-1]
			candidates = self.semantic_sim.get_post_event_instances(last_event, node.events)
			print(f"Candidates for expansion from '{last_event.split("/")[-1]}': {[candidate.split("/")[-1] for candidate in candidates]}")

			for candidate in candidates:
				new_events = node.events + [candidate]
				new_node = Node(events=new_events,
								parent=node,
								g=(node.g + 1) / self.max_events)
				new_node.h = 1 - state_similarity(new_node, query)
				new_node.f = new_node.g + new_node.h
				heapq.heappush(open_heap, (new_node.f, counter, new_node))
				print(f"New node added: events={new_node.get_event_names()}, g={new_node.g:.2f}, h={new_node.h:.2f}, f={new_node.f:.2f}")
				counter += 1
		
		print("No valid sequence found.")
		return None

In [125]:
constructive_adaptation = ConstructiveAdaptation(graph, semantic_sim, 15)

query = Query(initial_event_type="Conflict")

objective_node = constructive_adaptation.generate(query)

Initial node added: events=['antagonist_mocking_hero'], g=0.00, h=0.18, f=0.18
Initial node added: events=['antagonist_naps_during_race'], g=0.00, h=0.75, f=0.75
Initial node added: events=['magical_helper_grants_wish'], g=0.00, h=0.40, f=0.40
Initial node added: events=['hero_meets_his_soulmate'], g=0.00, h=0.96, f=0.96
Initial node added: events=['hero_passing_antagonist'], g=0.00, h=0.40, f=0.40
Initial node added: events=['villains_insult_hero'], g=0.00, h=0.73, f=0.73
Initial node added: events=['villain_attacks_flimsy_house'], g=0.00, h=0.01, f=0.01
Initial node added: events=['villain_attacks_medium_resistant_house'], g=0.00, h=0.89, f=0.89
Expanding node: events=['villain_attacks_flimsy_house'], g=0.00, h=0.01, f=0.01
Candidates for expansion from 'villain_attacks_flimsy_house': ['villains_insult_hero', 'villain_attacks_medium_resistant_house']
New node added: events=['villain_attacks_flimsy_house', 'villains_insult_hero'], g=0.07, h=0.79, f=0.86
New node added: events=['villai